# Notes on random stuff I read about...

## Heatmap Clustering

[post](http://nicolas.kruchten.com/content/2018/02/seriation/)

R package `seration`. Demo code [here](https://github.com/nicolaskruchten/seriation)

Talked about 3 methods:

1. Agglomerative clustering,
2. Optimal Leaf Ordering (starts with agglomerative clustering output then reorder branches of the dendrogram so as to minimize the sum of dissimilarities between adjacent leaves),
3. Traveling salesman (find the order of rows that minimizes the sum of dissmilarities, unconstrained by the clustering tree).

